In [1]:
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
from matplotlib import image
from scipy.spatial.distance import cdist
from PIL import Image
from tqdm import tqdm
import pandas as pd
from scipy import ndimage
import cv2

import warnings

warnings.filterwarnings("ignore")

In [2]:
root = "PUT YOUR PATH OF KVASIR-SEG DATASET HERE"
# example: root = "/home/yesindeed/Desktop/Kvasir-SEG"

# 1. Generate Heatmaps

We thank Yao et al [1] for their code of gaze heatmap generation.

[1] Rong, Yao, et al. "Human attention in fine-grained classification." BMVC 2021.

In [3]:
def gaussian(x, sx, y=None, sy=None):
    """Returns an array of np arrays (a matrix) containing values between
    1 and 0 in a 2D Gaussian distribution
    arguments
    x		-- width in pixels
    sx		-- width standard deviation
    keyword argments
    y		-- height in pixels (default = x)
    sy		-- height standard deviation (default = sx)
    """

    # square Gaussian if only x values are passed
    if y == None:
        y = x
    if sy == None:
        sy = sx
    # centers
    xo = x / 2
    yo = y / 2
    # matrix of zeros
    M = np.zeros([y, x], dtype=float)
    # gaussian matrix
    for i in range(x):
        for j in range(y):
            M[j, i] = np.exp(-1.0 * (((float(i) - xo) ** 2 / (2 * sx * sx)) + ((float(j) - yo) ** 2 / (2 * sy * sy))))

    return M


def normalize_map(s_map):
    norm_s_map = (s_map - np.min(s_map)) / ((np.max(s_map) - np.min(s_map)) * 1.0)
    return norm_s_map


def draw_heatmap(normed_gazepoints, originalsize, org_img_size=(768, 768)):
    heatmap = np.zeros(org_img_size, dtype=np.float32)

    for p in normed_gazepoints:
        x = round(p[0] * org_img_size[1]) - 1
        y = round(p[1] * org_img_size[0]) - 1

        heatmap[y, x] += p[2]
        # heatmap[y, x] += 1

    heatmap = ndimage.filters.gaussian_filter(heatmap, 70)
    heatmap = normalize_map(heatmap)
    heatmap = cv2.resize(heatmap, dsize=(originalsize[1], originalsize[0]), interpolation=cv2.INTER_CUBIC)

    return heatmap.astype(np.float16)

In [4]:
df_gaze = pd.read_csv("../../GazeMedSeg/kvasir_fixation.csv")

for i, (name, group) in enumerate(df_gaze.groupby("IMAGE")):
    img_file = group["IMAGE"].tolist()[0]
    org_img_size = (int(group["IMAGE_HEIGHT"].tolist()[0]), int(
        group["IMAGE_WIDTH"].tolist()[0]))

    gaze_list = np.array(
        df_gaze.loc[
            df_gaze["IMAGE"] == img_file,
            ["CURRENT_FIX_X", "CURRENT_FIX_Y", "CURRENT_FIX_DURATION"],
        ]
    )
    heatmap = draw_heatmap(gaze_list, org_img_size)

    img = Image.open(os.path.join(root, "images", img_file))
    gt = Image.open(os.path.join(root, "masks", img_file)).convert("L")

    gt_array = np.array(gt) / 255

    save_folder = os.path.join(root, "gaze", "heatmap")
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    heatmap_img = Image.fromarray((heatmap * 255).astype(np.uint8))
    heatmap_img.save(os.path.join(save_folder, img_file))

# 2. Refine Heatmap with CRF

In [5]:
import pydensecrf.densecrf as dcrf
from pydensecrf.utils import unary_from_softmax

In [6]:
def crf_inference(img, probs, t=10, scale_factor=1, compat=1.5):
    h, w = probs.shape

    probs = probs[None, :, :]
    probs = np.concatenate([1 - probs, probs], axis=0)

    d = dcrf.DenseCRF2D(w, h, 2)
    # unary = fake_prob.reshape((2, -1))
    unary = unary_from_softmax(probs)
    # unary = np.ascontiguousarray(unary)

    d.setUnaryEnergy(unary)
    # d.addPairwiseGaussian(3/scale_factor, compat=10)
    d.addPairwiseBilateral(sxy=80 / scale_factor, srgb=13, rgbim=np.copy(img), compat=compat)

    Q = d.inference(t)
    # crf = np.argmax(Q, axis=0).reshape((h, w))
    crf = np.array(Q)[1].reshape((h, w))

    return crf

In [8]:
scale_factor = 1
t = 10
compat = 1

for i, path in tqdm(enumerate(glob.glob(os.path.join(root, "gaze", "heatmap", "*.jpg")))):
    img_name = os.path.basename(path).split(".")[0]

    img = np.array(Image.open(os.path.join(
        root, "images", f"{img_name}.jpg")).convert("RGB"))
    gt = np.array(Image.open(os.path.join(
        root, "masks", f"{img_name}.jpg")).convert("L"))

    heatmap = Image.open(path).convert("L")
    heatmap = np.array(heatmap).astype(np.float32) / 255

    crf_map = crf_inference(
        img, heatmap, t=t, scale_factor=scale_factor, compat=compat)
    save_folder = os.path.join(root, "gaze", f"crf_compat{compat}")
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    crf_map_img = Image.fromarray((crf_map * 255).astype(np.uint8))
    crf_map_img.save(os.path.join(save_folder, f"{img_name}.jpg"))

1000it [06:38,  2.51it/s]


# 3. Assess Gaze Annotation Quality (Optional)

In [9]:
def compute_dice(pred, gt):
    inter = np.sum(pred * gt)

    dice = 2 * inter / (np.sum(pred) + np.sum(gt) + 1e-6)

    return dice


dice_crf_l = {}
dice_heatmap_l = {}

thres = 0.5

for i, path in tqdm(enumerate(glob.glob(os.path.join(root, "gaze", "heatmap", "*.jpg")))):
    path = os.path.basename(path).split(".")[0]
    gt = np.array(Image.open(os.path.join(
        root, "masks", f"{path}.jpg")).convert("L"))
    heatmap = np.array(Image.open(os.path.join(
        root, "gaze", "heatmap", f"{path}.jpg")).convert("L"))
    crf_map = np.array(Image.open(os.path.join(
        root, "gaze", f"crf_compat{compat}", f"{path}.jpg")).convert("L"))

    heatmap = heatmap.astype(np.float32) / 255
    crf_map = crf_map.astype(np.float32) / 255

    dice_heatmap = compute_dice(heatmap > thres, gt.astype(np.float32) / 255)
    dice_crf = compute_dice(crf_map > thres, gt.astype(np.float32) / 255)

    dice_crf_l[path] = dice_crf
    dice_heatmap_l[path] = dice_heatmap

print(f"Heatmap Dice: {np.mean(list(dice_heatmap_l.values()))}")
print(f"CRF Dice: {np.mean(list(dice_crf_l.values()))}")

1000it [00:07, 126.26it/s]

Heatmap Dice: 0.7449912705911774
CRF Dice: 0.7512073166932152
